In [1]:
import pandas as pd
import zarr

In [2]:
z_i = zarr.open('/workspaces/tmp_query_understanding_model/models/product_title_query_embedding/version_1/emb-epoch=12-step=120756--amazon-esci-title-query-uslocale--inputemb.zarr', 'r')
z_o = zarr.open('/workspaces/tmp_query_understanding_model/models/product_title_query_embedding/version_1/emb-epoch=12-step=120756--amazon-esci-title-query-uslocale--outputemb.zarr', 'r')

In [17]:
list(z_i.array_keys())

['dl-idx_b-idx_s-idx_pred_rank-0']

In [36]:
import torch
import numpy as np
from tqdm import tqdm

In [35]:
inds = [] 
sims = []
dszie = z_i['dl-idx_b-idx_s-idx_pred_rank-0'].shape[0]
for s in tqdm(range(0, dszie, 1000)):
    inds_i = z_i['dl-idx_b-idx_s-idx_pred_rank-0'][s:s+1000][:, 2]
    inds_o = z_o['dl-idx_b-idx_s-idx_pred_rank-0'][s:s+1000][:, 2]
    assert (inds_i == inds_o).all()
    emb_i = z_i['dl-idx_b-idx_s-idx_pred_rank-0'][s:s+1000][:, 3:]
    emb_o = z_o['dl-idx_b-idx_s-idx_pred_rank-0'][s:s+1000][:, 3:]
    inds.append(inds_i)
    sims_ = torch.nn.CosineSimilarity()(torch.tensor(emb_i).cuda(), torch.tensor(emb_o).cuda()).detach().cpu().numpy()
    sims.append(sims_)

100%|██████████| 426/426 [00:52<00:00,  8.04it/s]


In [38]:
inds = np.concatenate(inds).astype(int)

In [40]:
sims = np.concatenate(sims)

In [41]:
inds.shape, sims.shape

((425762,), (425762,))

In [46]:
import sys 
sys.path.append('../..')

In [49]:
from main_utils import LLM_EmbedManualData
data_module = LLM_EmbedManualData('/workspaces/tmp_query_understanding_model/datasets/product_title_query_embedding_manual/amazon-esci-title-query-uslocale.yaml')
data_module.hparams.raw_cache_dir_folder

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


'/data/junwang/.cache/general/35464de061558e33a7d10a7653930ea7'

In [50]:
!ls /data/junwang/.cache/general/35464de061558e33a7d10a7653930ea7

test.json.gz	   train.json.gz       val.json.gz
test.json.gz.lock  train.json.gz.lock  val.json.gz.lock


In [52]:
df = pd.read_json('/data/junwang/.cache/general/35464de061558e33a7d10a7653930ea7/test.json.gz', lines=True)

In [61]:
df['index'] = df.index.tolist()

In [62]:
assert (inds == df.index).all()

In [64]:
df['sims'] = sims

In [67]:
query_groups = df.groupby('text_output').agg({ 
    'labels': lambda x: [i for i in x], 
    'sims': lambda x: [i for i in x]
})

In [68]:
from sklearn.metrics import ndcg_score

In [74]:
ndcg_score(np.asarray([i['labels']]), np.asarray([i['sims']]))

0.8029482835498372

In [75]:
ndcgs = []
for i in tqdm(query_groups.to_dict('records')):
    ndcgs.append(ndcg_score(np.asarray([i['labels']]), np.asarray([i['sims']])))

100%|██████████| 22458/22458 [00:10<00:00, 2199.62it/s]


In [76]:
np.mean(ndcgs)

0.8766786853147283